<a href="https://www.kaggle.com/code/aadishchopra/disastertweets?scriptVersionId=124492953" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/saved_model.pb
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/keras_metadata.pb
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/assets/vocab.txt
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/variables/variables.index
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/variables/variables.data-00000-of-00001
/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
import datasets
import transformers
from transformers import AutoTokenizer,AutoModelForSequenceClassification,Trainer,TrainingArguments
from sklearn.model_selection import train_test_split

In [6]:
pd.set_option('display.max_colwidth', None)
df=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
#df=df[1:500]
#df_train,df_val=train_test_split(df,test_size=0.3)


,id,keyword,location,text,target
975,1411,body%20bag,NaN,One day this heart gone get me zipped up in a body bag.,0


In [7]:
df.sample()

,id,keyword,location,text,target
4422,6293,hostage,"Vancouver, British Columbia",Û÷RansomwareÛª holds B.C. manÛªs computer files hostage: A virus locked Andrew Wilson's family photos and otherÛ_ http://t.co/aQbLjComlN,0


# Remove punctuations, non-english characters
Û÷RansomwareÛª holds B.C. manÛªs computer files hostage: A virus locked Andrew Wilson's family photos and otherÛ_ http://t.co/aQbLjComlN	




In [28]:
#1. Remove characters other than english




Россия
💯
b'\\xdb'
b'\xc2\x89'
Â


In [5]:
pd.set_option('display.max_colwidth', None)
df_test=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

df_test=pd.DataFrame(df_test,columns=['id','text'])



df_test.head()

,id,text
0,0,Just happened a terrible car crash
1,2,"Heard about #earthquake is different cities, stay safe everyone."
2,3,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
3,9,Apocalypse lighting. #Spokane #wildfires
4,11,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
df_test_dict=datasets.Dataset.from_dict(df_test)

In [7]:
df=df[['text','target']]
df.head()
df_train,df_eval=train_test_split(df,test_size=0.1)

In [8]:
df_dict=datasets.Dataset.from_dict(df_train)
df_eval_dict=datasets.Dataset.from_dict(df_eval)

In [9]:
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')

In [10]:
def letstokenize(text):
    #set_trace()
    text_t=text['text']
    target=text['target']
    encoding=tokenizer(text_t,padding="max_length",truncation=True,max_length=128)
    #label=text['Sentiment']
    encoding["labels"]=target
    return encoding

encoded_df=df_dict.map(letstokenize)

encoded_eval_dict=df_eval_dict.map(letstokenize)

  0%|          | 0/6851 [00:00<?, ?ex/s]

  0%|          | 0/762 [00:00<?, ?ex/s]

In [11]:
disaster_bert=AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',
                                                             num_labels=2
                                                             )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [12]:
batch_size=8
metric_name='f1'
repo_name='kaggle-disaster-tweets'

args=TrainingArguments(
                  output_dir=repo_name,
                  evaluation_strategy='epoch',
                  metric_for_best_model=metric_name,
                  per_device_train_batch_size=batch_size,
                  num_train_epochs=4,
                  load_best_model_at_end=True,
                  #no_cuda=True,
                  per_device_eval_batch_size=batch_size,
                  weight_decay=0.05,
                  save_strategy='epoch',
                  learning_rate=2e-5,
                  report_to="none" 
                  #push_to_hub=True
                    )

In [13]:
from datasets import load_metric
from sklearn.metrics import roc_auc_score,f1_score,accuracy_score

In [14]:
#!pip install evaluate
import evaluate

In [15]:
def compute_metrics (eval_pred):
    metric = evaluate.load("f1")
    logits, labels = eval_pred
    preds = np.argmax(logits, axis = -1)
    return metric.compute(predictions=preds, references = labels)

In [16]:

trainer = Trainer(
    disaster_bert,
    args,
    train_dataset=encoded_df,
    eval_dataset=encoded_eval_dict,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
    )
     

In [17]:
#trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,0.470100,0.373904,0.825153
2,0.338100,0.471777,0.803768
3,0.270600,0.617170,0.788779
4,0.180300,0.727384,0.795632


TrainOutput(global_step=3428, training_loss=0.3068768246449278, metrics={'train_runtime': 480.5921, 'train_samples_per_second': 57.021, 'train_steps_per_second': 7.133, 'total_flos': 1802573840271360.0, 'train_loss': 0.3068768246449278, 'epoch': 4.0})

In [18]:
#trainer.evaluate()

{'eval_loss': 0.37390440702438354,
 'eval_f1': 0.8251533742331287,
 'eval_runtime': 4.0341,
 'eval_samples_per_second': 188.891,
 'eval_steps_per_second': 23.797,
 'epoch': 4.0}

In [19]:
'''
from evaluate import evaluator
task_evaluator = evaluator("text-classification")
# 1. Pass a model name or path
eval_results = task_evaluator.compute(model_or_pipeline="disaster_bert",data=df_test_dict,)
'''

'\nfrom evaluate import evaluator\ntask_evaluator = evaluator("text-classification")\n# 1. Pass a model name or path\neval_results = task_evaluator.compute(model_or_pipeline="disaster_bert",data=df_test_dict,)\n'

In [20]:
from transformers import TextClassificationPipeline
pipe = TextClassificationPipeline(model=disaster_bert, tokenizer=tokenizer, return_all_scores=False,device=0,)
# outputs a list of dicts like [[{'label': 'NEGATIVE', 'score': 0.0001223755971295759},  {'label': 'POSITIVE', 'score': 0.9998776316642761}]]


/opt/conda/lib/python3.7/site-packages/transformers/pipelines/text_classification.py:107: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  UserWarning,


In [21]:
df_test['result']=pipe(df_test['text'].tolist())

In [22]:
df_test['target']=df_test['result'].apply(lambda x: int(x['label'][-1]))

In [23]:
df_test

,id,text,result,target
0,0,Just happened a terrible car crash,"{'label': 'LABEL_1', 'score': 0.901736855506897}",1
1,2,"Heard about #earthquake is different cities, stay safe everyone.","{'label': 'LABEL_1', 'score': 0.9447317719459534}",1
2,3,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all","{'label': 'LABEL_1', 'score': 0.9658547639846802}",1
3,9,Apocalypse lighting. #Spokane #wildfires,"{'label': 'LABEL_1', 'score': 0.9766071438789368}",1
4,11,Typhoon Soudelor kills 28 in China and Taiwan,"{'label': 'LABEL_1', 'score': 0.9801334142684937}",1
...,...,...,...,...
3258,10861,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTENERS XrWn,"{'label': 'LABEL_1', 'score': 0.958180844783783}",1
3259,10865,Storm in RI worse than last hurricane. My city&amp;3others hardest hit. My yard looks like it was bombed. Around 20000K still without power,"{'label': 'LABEL_1', 'score': 0.966296374797821}",1
3260,10868,Green Line derailment in Chicago http://t.co/UtbXLcBIuY,"{'label': 'LABEL_1', 'score': 0.9781237244606018}",1
3261,10874,MEG issues Hazardous Weather Outlook (HWO) http://t.co/3X6RBQJHn3,"{'label': 'LABEL_1', 'score': 0.9755181074142456}",1


In [24]:
submit=df_test[['id','target']]

In [27]:
import shutil
#shutil.rmtree('/kaggle/working/kaggle-disaster-tweets')

In [28]:
os.remove('/kaggle/working/submissions.csv')

In [26]:
submit.to_csv('submission.csv',index=False)